In [1]:
!ls /kaggle/input


gru-cholec80  gru-ecrcd


In [2]:
# ====================================================
# 1️⃣ Imports & Device
# ====================================================
import torch
import torch.nn as nn
import numpy as np

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("✅ Using device:", DEVICE)


✅ Using device: cpu


In [3]:
# ====================================================
# 2️⃣ Define GRU Model Class
# ====================================================
class GRUModel(nn.Module):
    def __init__(self, input_dim, hidden=128, layers=2, out=3, dropout=0.3):
        super().__init__()
        self.gru = nn.GRU(
            input_dim, hidden, num_layers=layers, batch_first=True,
            dropout=dropout if layers > 1 else 0.0
        )
        self.head = nn.Sequential(nn.Dropout(0.3), nn.Linear(hidden, out))

    def forward(self, x):
        h, _ = self.gru(x)
        return self.head(h[:, -1, :])


In [4]:
import torch, torch.nn as nn

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class GRUModel(nn.Module):
    def __init__(self, input_dim, hidden=128, layers=2, out=3, dropout=0.3):
        super().__init__()
        self.gru = nn.GRU(
            input_dim, hidden, num_layers=layers, batch_first=True,
            dropout=dropout if layers > 1 else 0.0
        )
        self.head = nn.Sequential(nn.Dropout(0.3), nn.Linear(hidden, out))
    def forward(self, x):
        h,_ = self.gru(x); return self.head(h[:, -1])

# paths
ECRCD_PATH  = "/kaggle/input/gru-ecrcd/pytorch/default/1/gru_ecrcd.pth"
CHOLEC_PATH = "/kaggle/input/gru-cholec80/pytorch/default/1/gru_model.pth"

# input dims
ECRCD_INPUT_DIM  = 52
CHOLEC_INPUT_DIM = 9

# instantiate with correct hidden sizes
model_ecrcd  = GRUModel(ECRCD_INPUT_DIM, hidden=128, layers=2).to(DEVICE)
model_cholec = GRUModel(CHOLEC_INPUT_DIM, hidden=96,  layers=2).to(DEVICE)   # <-- fix

# load weights
model_ecrcd.load_state_dict(torch.load(ECRCD_PATH,  map_location=DEVICE))
model_cholec.load_state_dict(torch.load(CHOLEC_PATH, map_location=DEVICE))

model_ecrcd.eval(); model_cholec.eval()
print("✅ Loaded both models with matching architectures on", DEVICE)


✅ Loaded both models with matching architectures on cpu


In [5]:
# ====================================================
# 4️⃣ Hybrid Fusion Model (Late Fusion)
# ====================================================
class HybridFusionModel(nn.Module):
    def __init__(self, model_ecrcd, model_cholec, fusion="weighted", w_e=0.5):
        super().__init__()
        self.model_ecrcd = model_ecrcd
        self.model_cholec = model_cholec
        self.fusion = fusion
        self.w_e = w_e
        self.sigmoid = nn.Sigmoid()

    def forward(self, x_ecrcd, x_cholec):
        with torch.no_grad():
            out_e = self.sigmoid(self.model_ecrcd(x_ecrcd))
            out_c = self.sigmoid(self.model_cholec(x_cholec))
        if self.fusion == "weighted":
            out = self.w_e * out_e + (1 - self.w_e) * out_c
        else:
            out = torch.cat([out_e, out_c], dim=1)
        return out

# ✅ Instantiate hybrid model
hybrid_model = HybridFusionModel(model_ecrcd, model_cholec, fusion="weighted", w_e=0.6)
print("✅ Hybrid model ready for inference")


✅ Hybrid model ready for inference


In [6]:
# ====================================================
# 5️⃣ Example Hybrid Inference
# ====================================================
# Simulated input tensors (batch=2, sequence=300)
x_ecrcd = torch.randn(2, 300, ECRCD_INPUT_DIM).to(DEVICE)
x_cholec = torch.randn(2, 300, CHOLEC_INPUT_DIM).to(DEVICE)

with torch.no_grad():
    preds = hybrid_model(x_ecrcd, x_cholec)

print("🔹 Hybrid output shape:", preds.shape)
print("🔹 Sample predictions:", preds)


🔹 Hybrid output shape: torch.Size([2, 3])
🔹 Sample predictions: tensor([[0.1340, 0.3514, 0.6824],
        [0.9029, 0.8104, 0.8577]])
